# Data Processing Functions for AvAnt2025_AnalysisEKG

In [ ]:
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame as DF

import seaborn as sns
sns.set(color_codes=True)
#np.random.seed(sum(map(ord, "distributions")))
from sklearn import linear_model  # packages for the logistic regression function to plot the logistic regression 
from sklearn.linear_model import LogisticRegression # packages for the logistic regression function to plot the logistic regression 
import scipy
from scipy import stats, integrate
from scipy.stats import mode
from scipy.stats.stats import pearsonr # Pearson's correlation
from scipy.stats import sem
from copy import copy as copy
import operator as operator


# Plotting tools
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from functools import reduce

%pylab inline
figsize(5, 5)
import seaborn as sns

import os

from ecgdetectors import Detectors

# Added to avoid OMP:error#15
os.environ['KMP_DUPLICATE_LIB_OK']='True'

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/var/folders/_k/84ry5gsj3bnd86w91nlcyck00000gr/T/ipykernel_28427/3025049525.py:13: DeprecationWarning: Please import `pearsonr` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.stats.stats import pearsonr # Pearson's correlation
/opt/anaconda3/lib/python3.12/site-packages/IPython/core/magics/pylab.py:166: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [ ]:
#set sample rate
sampleRate = 100
fs = sampleRate
detectors = Detectors(sampleRate)

# Preprocess data & detect r-peaks

### Use ECG Detectors (two average detector) for r-peak detection

In [ ]:
###Uses pank tompkins detector for r peak detection


def preprocess(delaydata):
    """
    preprocess data by adding r-peak information
    parameters (delaydata): the full preprocessed dataframe
    returns dataframe with added columns:
        pEKG (raw ekg data)
        peak (boolean value of wether a peak is detected at the current row)
        peak_time: the time at which peak occurs
    """
    delayProcessed = pd.DataFrame()

    for participant in np.unique(delaydata['partNum']):
        dataPart = delaydata[delaydata['partNum'] == participant].copy()
        time =dataPart['time'].values
        signal = dataPart.EKG.values 
        r_peaks = detectors.pan_tompkins_detector(signal)
        peaks_mask = np.zeros_like(signal, dtype=bool)
        peaks_mask[r_peaks] = True

        dataPart['pEKG'] = signal #same as raw EKG value, changing the name for consistency purposes
        dataPart['peak'] = peaks_mask
        dataPart['peak_time'] = np.where(peaks_mask, time, np.nan)

        delayProcessed = pd.concat([delayProcessed, dataPart], ignore_index=True)

    return delayProcessed

# extract baseline using the start trial period (first 2 seconds)

In [ ]:
def IBI_baseline(mergedDFClean):
    dataBaseline = mergedDFClean
    dataBaseline0= preprocess(dataBaseline)

    ### create baseline IBI for each trial

    rows = []

    for participant in dataBaseline0['partNum'].unique():
        part_data = dataBaseline0[dataBaseline0['partNum'] == participant]
        for session in dataBaseline0['session'].unique():
            sess_data = part_data[part_data['session'] == session]
            
            #compute baseline IBI at trial level
            for trial in sess_data['trials'].unique():
                trial_data = sess_data[sess_data['trials'] == trial]
                part_trial = trial_data['PART_trial'].iloc[0]
                delay      = trial_data['delay_time'].iloc[0]

                
                peak_idxs = np.where(trial_data['peak'])[0]        
                
                
                if len(peak_idxs) < 2:
                    mean_ibi = np.nan
                
                #divide ibi values by sample rate
                else:
                    ibi_samples = np.diff(peak_idxs)
                    mean_ibi = ibi_samples.mean()/ sampleRate

                rows.append({
                    'participant': participant,
                    'session': session,
                    'trial': trial,
                    'delay_time':  delay,
                    'baseline': mean_ibi,
                    'PART_trial': part_trial
                    
                })
            
    #return baseline_IBI
    baseline_IBI = pd.DataFrame(rows)

    
    return baseline_IBI


# 1. Compute IBI intervals from before infochoice presentation till after stimulus presentation time


In [ ]:
# compute IBI values by picking time windows 2s before info cue onset till 4s after stimulus onset, 
# normalize by subtracting baseline IBI values on the trial level
# Parameters: data (dataframe to be computed IBI for); baseline_df (baseline values for this same set of data)

def compute_IBI (data,baseline_df):

    """
    Returns a DataFrame with one row per trial, containing:
      - participant, delay_time, PART_trial
      - IBIs: list of normalized IBIs (s)
      - time_norm: list of IBI timestamps relative to window start (s)
      - cue_norm, stim_norm: cue and stim onsets relative to window start (s)
      *** remove outliers with IBI values > 2 (some trials have recording issues)
    """
    rows = []

    for trials in data['PART_trial'].unique():
        trial_data = data[data['PART_trial']==trials]
        participant = trial_data['partNum'].iloc[0]
        delay = trial_data['delay_time'].iloc[0]

        #extract time point stimInfo or stimNoInfo is presented
        if 'stimInfoOnset' in trial_data.columns and trial_data['stimInfoOnset'].eq(1).any():
            cueTime = trial_data.loc[trial_data['stimInfoOnset']==1, 'time'].iloc[0]
        elif 'stimNoInfoOnset' in trial_data.columns and trial_data['stimNoInfoOnset'].eq(1).any():
            cueTime = trial_data.loc[trial_data['stimNoInfoOnset']==1, 'time'].iloc[0]
        else:
            cueTime = np.nan

        #extract time point scream or noScream is presented
        if 'screamOnset' in trial_data.columns and trial_data['screamOnset'].eq(1).any():
            stimTime = trial_data.loc[trial_data['screamOnset']==1,'time'].iloc[0]
        elif 'noScreamOnset' in trial_data.columns and trial_data['noScreamOnset'].eq(1).any():
            stimTime = trial_data.loc[trial_data['noScreamOnset']==1,'time'].iloc[0]    
        else:
            stimTime = np.nan

        #define start and end time of where ibi values will be extracted
        pre_window = 2.0 #adding 2 sec before infocue onset
        post_window = 4.0 #adding 4 sec after outcome stimulus presentation (outcome pres + end trial time)
        start = cueTime - pre_window
        end = stimTime + post_window

        
        # restrict to peaks in that window
        peak_times = trial_data.loc[trial_data['peak']==1, 'time'].values
        in_window = peak_times[(peak_times >= start) & (peak_times <= end)]
        if len(in_window) < 2:
            continue
        
        #compute raw and normalized ibis
        ibis      = np.diff(in_window)
        ibi_times = (in_window[:-1] + in_window[1:]) / 2

        # —— REMOVE OUTLIERS: drop any IBI > 2 seconds —— 
        mask = ibis < 2.0
        ibis = ibis[mask]
        ibi_times = ibi_times[mask]
        if len(ibis) == 0:
            # nothing left after filtering
            continue

        # —— normalize time to window start —— 
        t_norm     = (ibi_times - start).tolist()
        cue_norm   = cueTime   - start
        stim_norm  = stimTime  - start


        #normalize IBI by trial-level baseline 
        base       = baseline_df.loc[
                         baseline_df['PART_trial']==trials,'baseline'
                     ].iloc[0]
        ibis_norm  = (ibis - base).tolist()

        
        delay = trial_data['delay_time'].iloc[0]

        
        # collect one row per IBI
        rows.append({
            'participant': participant,
            'delay_time':  delay,
            'PART_trial':  trials,
            'IBIs':        ibis_norm,
            'time_norm':   t_norm,
            'cue_norm':    cue_norm,
            'stim_norm':   stim_norm
        })
        
    return pd.DataFrame(rows)

        
    

In [ ]:
# Averaging IBI values by trial and participant levels, for windows of entire delay time
# parameters: trial_df(data frame with computed IBI values); baseline_df(data frame with IBI baseline values)

def avg_IBI(trial_df,baseline_df):
    """
    From trial_df (output of compute_IBI), compute:
      1) per-trial mean IBI between infoOnset and stimOnset
      2) per-participant x delay mean of those trial means
      3) per-delay mean across participants.

    Returns
    -------
    trial_df2: DataFrame with columns 
        participant, PART_trial, delay_time, mean_IBI on trial level
    part_delay_df : DataFrame with columns
        participant, PART_trial, delay_time, mean_IBI on participant level
    delay_df : DataFrame with columns
        delay_time, mean_IBI (1 value for each delay)
    """
    # 1) compute per‐trial means
    trial_means = []
    for _, row in trial_df.iterrows():

        ibis  = row['IBIs']
        times = row['time_norm']
        c, s  = row['cue_norm'], row['stim_norm']

        # select only the IBIs whose midpoint lies between cue and stim
        sel   = [ibi for ibi, t in zip(ibis, times) if c <= t <= s]
        if not sel:
            continue

        mean_norm    = np.mean(sel)
        baseline     = baseline_df.loc[
                           baseline_df['PART_trial']==row['PART_trial'],
                           'baseline'
                       ].iloc[0]
        
        #adding back baseline value to show baseline vs. raw EKG differences
        mean_raw     = mean_norm + baseline

        trial_means.append({
            'participant': row['participant'],
            'PART_trial': row['PART_trial'],
            'delay_time':  row['delay_time'],
            'mean_IBI':    mean_raw
        })

    trial_df2 = pd.DataFrame(trial_means)

    # 2) average by participant x delay
    part_delay_df = (
        trial_df2
        .groupby(['participant','delay_time'], as_index=False)
        ['mean_IBI']
        .mean()
    )

    # 3) average across participants, per delay
    delay_df = (
        part_delay_df
        .groupby('delay_time', as_index=False)
        ['mean_IBI']
        .mean()
    )

    return trial_df2,part_delay_df, delay_df


In [ ]:
#plot changes from baseline IBI to average IBI across delay time

def plot_baseline_raw_ibi(
    baseline1, baseline2, baseline3,
    raw_part1, raw_part2, raw_part3,
    cond_labels=('Info+Scream','Info+NoScream','NoInfo')
):
    """
    Plot paired baseline vs. raw dots with error bars for each condition,
    one figure per delay.

    baseline ibi value vs. average ibi value during the entire delay period 

    Parameters
    ----------
    baselineX : pd.DataFrame with columns ['participant','delay_time','baseline']
        Trial-level baseline IBIs.
    raw_partX : pd.DataFrame with columns ['participant','delay_time','mean_IBI']
        Participant-level raw IBI means (first output of avg_IBI).
    cond_labels : tuple of 3 condition names in the order of dataframes.
    """
    # --- Summarize baseline: trials→participant→delay ---
    def summarize_baseline(df, label):
        part = df.groupby(['participant','delay_time'], as_index=False)['baseline'].mean()
        stats = part.groupby('delay_time')['baseline'].agg(['mean','std','count']).reset_index()
        stats['sem'] = stats.apply(
            lambda r: (r['std'] / np.sqrt(r['count'])) if r['count'] >= 2 else 0.0,
            axis=1
        )
        stats = stats.rename(columns={'mean': f'{label}_mean', 'sem': f'{label}_sem'})
        print(stats)
        return stats[['delay_time', f'{label}_mean', f'{label}_sem']]

    # --- Summarize raw: participant→delay ---
    def summarize_raw_part(df, label):
        part = df.groupby(['participant','delay_time'], as_index=False)['mean_IBI'].mean()
        stats = part.groupby('delay_time')['mean_IBI'].agg(['mean','std','count']).reset_index()
        stats['sem'] = stats.apply(
            lambda r: (r['std'] / np.sqrt(r['count'])) if r['count'] >= 2 else 0.0,
            axis=1
        )
        stats = stats.rename(columns={'mean': f'{label}_raw_mean', 'sem': f'{label}_raw_sem'})
        print(stats)
        return stats[['delay_time', f'{label}_raw_mean', f'{label}_raw_sem']]

    # baseline summaries
    b1 = summarize_baseline(baseline1, cond_labels[0])
    b2 = summarize_baseline(baseline2, cond_labels[1])
    b3 = summarize_baseline(baseline3, cond_labels[2])
    # raw summaries at participant level
    r1 = summarize_raw_part(raw_part1, cond_labels[0])
    r2 = summarize_raw_part(raw_part2, cond_labels[1])
    r3 = summarize_raw_part(raw_part3, cond_labels[2])

    # merge all on delay_time
    merged = b1.merge(b2, on='delay_time').merge(b3, on='delay_time')
    merged = merged.merge(r1, on='delay_time').merge(r2, on='delay_time').merge(r3, on='delay_time')


    # plotting
    x = np.arange(len(cond_labels))
    offset = 0.05
    for _, row in merged.iterrows():
        delay = row['delay_time']
        base_means = [row[f'{c}_mean'] for c in cond_labels]
        base_sems  = [row[f'{c}_sem']  for c in cond_labels]
        raw_means  = [row[f'{c}_raw_mean'] for c in cond_labels]
        raw_sems   = [row[f'{c}_raw_sem']  for c in cond_labels]

        fig, ax = plt.subplots(figsize=(6,4))
        ax.errorbar(x - offset, base_means, yerr=base_sems,
                    fmt='o', color='C0', label='Baseline', capsize=4)
        ax.errorbar(x + offset, raw_means, yerr=raw_sems,
                    fmt='o', color='C1', label='Raw', capsize=4)
        for xi, b, r in zip(x, base_means, raw_means):
            ax.plot([xi - offset, xi + offset], [b, r],
                    color='gray', linestyle='--', linewidth=1)

        ax.set_xticks(x)
        ax.set_xticklabels(cond_labels)
        ax.set_xlabel('Condition')
        ax.set_ylabel('Mean IBI (s)')
        ax.set_title(f'Change Baseline→Anticipation IBI — Delay {delay}s')
        ax.legend()
        ax.grid(axis='y', linestyle=':', alpha=0.7)
        plt.tight_layout()
        plt.show()

    return merged


In [ ]:
#compute difference in ibi between delay time average and baseline value at trial level, 
# return participant level df and trial level df

def compute_diff_ibi(
    baseline_dfs, raw_trial_dfs, cond_labels, outlier_thresh=3.0
):
    """
    1) Merge trial-level baseline & raw → compute diff per trial
    2) Remove trial diffs beyond ±3 sigma per delay
    3) Average remaining diffs → participant-level diff
    Returns: diff_part_df, diff_trial_df
    """
    all_trials = []
    all_parts  = []

    for b_df, r_df, cond in zip(baseline_dfs, raw_trial_dfs, cond_labels):
        trial = (
            pd.merge(
                b_df[['participant','delay_time','PART_trial','baseline']],
                r_df[['participant','delay_time','PART_trial','mean_IBI']],
                on=['participant','delay_time','PART_trial']
            )
            .assign(diff=lambda df: df['baseline'] - df['mean_IBI'],
                    condition=cond)
        )

        # filter outliers with threshold of 3 sigma per delay
        def _filter(df):
            mu, sigma = df['diff'].mean(), df['diff'].std()
            return df[df['diff'].between(mu - outlier_thresh*sigma, mu + outlier_thresh*sigma)]

        trial_clean = trial.groupby('delay_time', group_keys=False).apply(_filter)
        all_trials.append(trial_clean)

        #participant-level average diff
        part = (
            trial_clean
            .groupby(['participant','delay_time','condition'], as_index=False)['diff']
            .mean()
        )
        all_parts.append(part)

    diff_trial_df = pd.concat(all_trials, ignore_index=True)
    diff_part_df  = pd.concat(all_parts,  ignore_index=True)
    return diff_part_df, diff_trial_df



#### Plot ibi difference by delay

In [ ]:
#after compute_diff_ibi, collapse delay condition,
# plot ibi difference at participant level
# each condition is plotted, averaged across all delays

def plot_overall_diff_ibi(diff_part_df, cond_labels):
    """
    Takes participant-level diffs, computes mean±SEM per delay & condition,
    and plots one error-bar figure across delays.
    """
    x = np.arange(len(cond_labels))

    # compute stats
    stats = (
        diff_part_df
        .groupby('condition')['diff_z']
        .agg(['mean','std','count'])
        .reindex(cond_labels)        # ensure order
    )
    stats['sem'] = stats['std'] / np.sqrt(stats['count'])

    means = stats['mean'].values
    sems  = stats['sem'].values

    plt.figure(figsize=(6,4))
    plt.errorbar(
        x, means, yerr=sems,
        fmt='o', capsize=4, linewidth=1.5
    )
    plt.xticks(x, cond_labels)
    plt.xlabel('Condition')
    plt.ylabel('Mean(baseline-raw) IBI (s)')
    plt.title(f'IBI Change by Condition')
    plt.grid(axis='y', linestyle=':', alpha=0.7)
    plt.tight_layout()
    plt.show()

In [ ]:
#after compute_diff_ibi, plot ibi difference at participant level, 
# separated by delay time
# each condition with each deplay is plotted
def plot_diff_ibi(diff_part_df, cond_labels):
    """
    Takes participant-level diffs, computes mean±SEM per delay & condition,
    and plots one error-bar figure per delay.
    """
    x = np.arange(len(cond_labels))

    for delay, grp in diff_part_df.groupby('delay_time'):
        # compute stats
        stats = (
            grp
            .groupby('condition')['diff_z']
            .agg(['mean','std','count'])
            .reindex(cond_labels)        # ensure order
        )
        stats['sem'] = stats['std'] / np.sqrt(stats['count'])

        means = stats['mean'].values
        sems  = stats['sem'].values

        plt.figure(figsize=(6,4))
        plt.errorbar(
            x, means, yerr=sems,
            fmt='o', capsize=4, linewidth=1.5
        )
        plt.xticks(x, cond_labels)
        plt.ylim(-0.6,0.45)
        plt.xlabel('Condition')
        plt.ylabel('Mean(baseline-raw) IBI (s)')
        plt.title(f'IBI Change by Condition — Delay {delay}s')
        plt.grid(axis='y', linestyle=':', alpha=0.7)
        plt.tight_layout()
        plt.show()
    

In [ ]:
#after compute_diff_ibi, group delay condition by long vs. short delay (5s,10s vs. 20s, 40s)
# plot ibi difference at participant level
# each condition is plotted, for the two delay groups
def plot_diff_ibi_groupedDelay(diff_part_df, cond_labels):
    """
    For each of two delay groups (Short: 5,10s; Long: 20,40s):
      - Compute mean±SEM of diff_z per condition (across all those delays)
      - Plot side-by-side errorbar panels
    """
    # define delay groups
    groups = {
        'Short delay (5-10 s)': [5.0, 10.0],
        'Long delay (20-40 s)': [20.0, 40.0]
    }
    x = np.arange(len(cond_labels))
    
    fig, (ax_short, ax_long) = plt.subplots(1, 2, figsize=(12, 4), sharey=True)
    
    for ax, (grp_name, delays) in zip(
        (ax_short, ax_long),
        groups.items()
    ):
        df_grp = diff_part_df[diff_part_df['delay_time'].isin(delays)]
        
        # aggregate per condition
        stats = (
            df_grp
            .groupby('condition')['diff_z']
            .agg(['mean','std','count'])
            .reindex(cond_labels)
        )
        stats['sem'] = stats['std'] / np.sqrt(stats['count'])
        
        # plot
        ax.errorbar(
            x, stats['mean'], yerr=stats['sem'],
            fmt='o', capsize=5, linewidth=1.5
        )
        ax.set_xticks(x)
        ax.set_xticklabels(cond_labels)
        ax.set_title(grp_name)
        ax.set_xlabel('Condition')
        ax.grid(axis='y', linestyle=':', alpha=0.7)
    
    ax_short.set_ylabel('Mean(diff_z) IBI (s)')
    plt.tight_layout()
    plt.show()


#### Plot ibi difference by condition

In [ ]:
#plot ibi difference by different conditions
#x-axis represents different delay time

def plot_delay_effect_byCond(diff_part_df, cond_labels):
    """
    For each condition, compute mean±SEM of diff_z across delay_time 
    and plot them in one error-bar plot.
    
    Parameters
    ----------
    diff_part_df : pd.DataFrame with columns ['participant','delay_time','condition','diff_z']
    cond_labels  : sequence of condition names to plot (and order)
    """
    # 1) get sorted unique delays and an x‐axis
    delays = sorted(diff_part_df['delay_time'].unique())
    x = np.arange(len(delays))

    # 2) one figure per condition
    for cond in cond_labels:
        grp = diff_part_df[diff_part_df['condition'] == cond]

        # 3) compute mean, std, count, sem per delay_time
        stats = (
            grp
            .groupby('delay_time')['diff_z']
            .agg(['mean','std','count'])
            .reindex(delays)  # ensure same order
        )
        stats['sem'] = stats['std'] / np.sqrt(stats['count'])

        # 4) plot
        plt.figure(figsize=(6,4))
        plt.errorbar(
            x, 
            stats['mean'].values, 
            yerr=stats['sem'].values,
            fmt='o', capsize=4, linewidth=1.5
        )
        plt.xticks(x, delays)
        plt.xlabel('Delay Time (s)')
        plt.ylabel(f'Mean diff_z (baseline-raw IBI) — {cond}')
        plt.title(f'Delay Effect for {cond}')
        plt.grid(axis='y', linestyle=':', alpha=0.7)
        plt.tight_layout()
        plt.show()


In [ ]:
#plot ibi difference averaging all conditions, see only delay effect

def plot_delay_effect(diff_part_df):
    """
    Plot overall delay effect collapsing all conditions
    
    Parameters
    ----------
    diff_part_df : pd.DataFrame with columns ['participant','delay_time','condition','diff_z']
    """
    # 1) get sorted unique delays and an x‐axis
    delays = sorted(diff_part_df['delay_time'].unique())
    x = np.arange(len(delays))


    # 3) compute mean, std, count, sem per delay_time
    stats = (
        diff_part_df
        .groupby('delay_time')['diff_z']
        .agg(['mean','std','count'])
        .reindex(delays)  # ensure same order
        )
    stats['sem'] = stats['std'] / np.sqrt(stats['count'])

    # 4) plot
    plt.figure(figsize=(6,4))
    plt.errorbar(
        x, 
        stats['mean'].values, 
        yerr=stats['sem'].values,
        fmt='o', capsize=4, linewidth=1.5
    )
    plt.xticks(x, delays)
    plt.xlabel('Delay Time (s)')
    plt.ylabel(f'Mean diff IBI (baseline-raw IBI)')
    plt.title(f'Overall Delay Effect')
    plt.grid(axis='y', linestyle=':', alpha=0.7)
    plt.tight_layout()
    plt.show()


### Mis Function to See Data Status

In [ ]:
#debug to see ibi data distribution through box plots

def plot_change_dots_with_error(
    baseline1, baseline2, baseline3,
    raw_part1, raw_part2, raw_part3,
    cond_labels=('Info+Scream','Info+NoScream','NoInfo')
):
    """
    Plot paired baseline vs. raw dots with error bars for each condition,
    one figure per delay.

    Parameters
    ----------
    baselineX : pd.DataFrame with columns ['participant','delay_time','baseline']
        Trial-level baseline IBIs.
    raw_partX : pd.DataFrame with columns ['participant','delay_time','mean_IBI']
        Participant-level raw IBI means (first output of avg_IBI).
    cond_labels : tuple of 3 condition names in the order of dataframes.
    """
    # --- Summarize baseline: trials→participant→delay ---
    def summarize_baseline(df, label):
        part = df.groupby(['participant','delay_time'], as_index=False)['baseline'].mean()
        stats = part.groupby('delay_time')['baseline'].agg(['mean','std','count']).reset_index()
        stats['sem'] = stats.apply(
            lambda r: (r['std'] / np.sqrt(r['count'])) if r['count'] >= 2 else 0.0,
            axis=1
        )
        stats = stats.rename(columns={'mean': f'{label}_mean', 'sem': f'{label}_sem'})
        print(stats)
        return stats[['delay_time', f'{label}_mean', f'{label}_sem']]

    # --- Summarize raw: participant→delay ---
    def summarize_raw_part(df, label):
        part = df.groupby(['participant','delay_time'], as_index=False)['mean_IBI'].mean()
        stats = part.groupby('delay_time')['mean_IBI'].agg(['mean','std','count']).reset_index()
        stats['sem'] = stats.apply(
            lambda r: (r['std'] / np.sqrt(r['count'])) if r['count'] >= 2 else 0.0,
            axis=1
        )
        stats = stats.rename(columns={'mean': f'{label}_raw_mean', 'sem': f'{label}_raw_sem'})
        print(stats)
        return stats[['delay_time', f'{label}_raw_mean', f'{label}_raw_sem']]

    # baseline summaries
    b1 = summarize_baseline(baseline1, cond_labels[0])
    b2 = summarize_baseline(baseline2, cond_labels[1])
    b3 = summarize_baseline(baseline3, cond_labels[2])
    # raw summaries at participant level
    r1 = summarize_raw_part(raw_part1, cond_labels[0])
    r2 = summarize_raw_part(raw_part2, cond_labels[1])
    r3 = summarize_raw_part(raw_part3, cond_labels[2])

    # merge all on delay_time
    merged = b1.merge(b2, on='delay_time').merge(b3, on='delay_time')
    merged = merged.merge(r1, on='delay_time').merge(r2, on='delay_time').merge(r3, on='delay_time')

    # plotting
    x = np.arange(len(cond_labels))
    offset = 0.15
    widths = 0.25

    for _, row in merged.iterrows():
        delay = row['delay_time']

        # collect the raw arrays for this delay
        base_data = [
            baseline1.loc[baseline1['delay_time']==delay, 'baseline'],
            baseline2.loc[baseline2['delay_time']==delay, 'baseline'],
            baseline3.loc[baseline3['delay_time']==delay, 'baseline'],
        ]
        raw_data = [
            raw_part1.loc[raw_part1['delay_time']==delay, 'mean_IBI'],
            raw_part2.loc[raw_part2['delay_time']==delay, 'mean_IBI'],
            raw_part3.loc[raw_part3['delay_time']==delay, 'mean_IBI'],
        ]

        fig, ax = plt.subplots(figsize=(6,4))

        # boxplots for baseline (blue) & raw (orange)
        bp1 = ax.boxplot(
            base_data,
            positions = x - offset,
            widths    = widths,
            patch_artist=True,
            boxprops  = dict(facecolor='C0', alpha=0.3),
            medianprops=dict(color='C0'),
            whiskerprops=dict(color='C0'),
            capprops   =dict(color='C0'),
            flierprops =dict(markeredgecolor='C0')
        )
        bp2 = ax.boxplot(
            raw_data,
            positions = x + offset,
            widths    = widths,
            patch_artist=True,
            boxprops  = dict(facecolor='C1', alpha=0.3),
            medianprops=dict(color='C1'),
            whiskerprops=dict(color='C1'),
            capprops   =dict(color='C1'),
            flierprops =dict(markeredgecolor='C1')
        )

        # overlay the means as dots if you like
        base_means = [row[f'{c}_mean'] for c in cond_labels]
        raw_means  = [row[f'{c}_raw_mean'] for c in cond_labels]
        ax.scatter(x - offset, base_means, color='C0', s=50, zorder=3)
        ax.scatter(x + offset, raw_means,  color='C1', s=50, zorder=3)

        ax.set_xticks(x)
        ax.set_xticklabels(cond_labels)
        ax.set_xlabel('Condition')
        ax.set_ylabel('Mean IBI (s)')
        ax.set_title(f'Change Baseline→Raw IBI — Delay {delay}s')

        # legend hack
        ax.plot([], [], color='C0', label='Baseline')
        ax.plot([], [], color='C1', label='Raw')
        ax.legend()

        ax.grid(axis='y', linestyle=':', alpha=0.7)
        plt.tight_layout()
        plt.show()
